# Test nat_zacros package

## Import packages and get path to Zacros calculations

In [1]:
import os
import platform 
from pathlib import Path
from nat_zacros import lattice, state
import numpy as np

# Dictionary mapping usernames to data paths
user_paths = {
    'a-DJA'  : Path('c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations'),
    'akandra': Path('/home/akandra/O_Pt111/zacros_calculations'),
    # Add more collaborators here as needed
    # note that file names and paths are case-sensitive here, even on Windows because file existence is checked in python
    # in a case sensitive manner
}

if platform.system() == 'Windows':
    username = os.getenv('USERNAME')
else:  # Linux/Mac
    username = os.getenv('USER')

# Get path for current user
if username in user_paths:
    userpath = user_paths[username]
else:
    raise ValueError(f"Unknown user: {username}. Please add your username and path to user_paths dictionary.")

print(f"Operating System : {platform.system()}")
print(f"username         : {username}")
print(f"userpath         : {userpath.as_posix()}")
# Show the type and value
print(f"repr(userpath)   : {repr(userpath)}")

Operating System : Windows
username         : a-DJA
userpath         : c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations
repr(userpath)   : WindowsPath('c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations')


In [2]:
lat = lattice(userpath / 'fn_3leed/jobs/1/traj_1')
lat.site_coordinations
print( f'len(lat) = {len(lat)}' )

len(lat) = 1600


In [3]:
# create state object with empty occupation
ads = state(len(lat))
print(f'occupation empty lattice      = {repr(ads.occupation)} with '
      f'{np.count_nonzero(ads.occupation):3d} nonzero elements')
# load occupation from file
ads.get_state(userpath / 'fn_3leed/jobs/1/traj_1', idx=0)
print(f'occupation after get_lattice  = {repr(ads.occupation)} with '
      f'{np.count_nonzero(ads.occupation):3d} nonzero elements' )

occupation empty lattice      = array([0, 0, 0, ..., 0, 0, 0], shape=(1600,)) with   0 nonzero elements
occupation after get_lattice  = array([0, 0, 0, ..., 0, 0, 0], shape=(1600,)) with 176 nonzero elements


## Test new state methods

In [4]:
# Test coverage and count methods
print(f"State representation: {repr(ads)}")
print(f"\nCoverage: {ads.get_coverage():.4f}")
print(f"Number of adsorbates: {ads.get_n_adsorbates()}")
print(f"Total sites: {ads.nsites}")

State representation: state(nsites=1600, n_adsorbates=176, coverage=0.110)

Coverage: 0.1100
Number of adsorbates: 176
Total sites: 1600


In [5]:
# Load a different state and compare
ads2 = state(len(lat))
ads2.get_state(userpath / 'fn_3leed/jobs/1/traj_1', idx=10)  # Load state at idx=10

print(f"\nState at idx=0:  {repr(ads)}")
print(f"State at idx=10: {repr(ads2)}")
print(f"\nCoverage change: {ads2.get_coverage() - ads.get_coverage():.4f}")


State at idx=0:  state(nsites=1600, n_adsorbates=176, coverage=0.110)
State at idx=10: state(nsites=1600, n_adsorbates=176, coverage=0.110)

Coverage change: 0.0000


In [6]:
# Test coordinate extraction (composition: state USES lattice)
occupied_coords = ads.get_occupied_coords(lat)

print(f"\nOccupied coordinates shape: {occupied_coords.shape}")
print(f"First 5 occupied coordinates:\n{occupied_coords[:5]}")


Occupied coordinates shape: (176, 2)
First 5 occupied coordinates:
[[ 7.0528375  10.58708984]
 [11.28454    17.91661357]
 [14.105675   22.80296272]
 [16.92681    27.68931188]
 [21.1585125  35.01883561]]


In [7]:
# Test site query methods
occupied_sites = ads.get_occupied_sites()
empty_sites = ads.get_empty_sites()

print(f"\nOccupied sites (first 10): {occupied_sites[:10]}")
print(f"Number of occupied sites: {len(occupied_sites)}")
print(f"Number of empty sites: {len(empty_sites)}")
print(f"Total check: {len(occupied_sites) + len(empty_sites)} == {ads.nsites}")


Occupied sites (first 10): [ 4  7  9 11 14 35 46 59 72 73]
Number of occupied sites: 176
Number of empty sites: 1424
Total check: 1600 == 1600


In [ ]:
# # Reload module to get fixed PBC methods
# import importlib
# import nat_zacros
# importlib.reload(nat_zacros)

# # Recreate lattice object with updated class
# lat = nat_zacros.lattice(userpath / 'fn_3leed/jobs/1/traj_1')
# print("Module reloaded, lattice recreated")

Module reloaded, lattice recreated


In [9]:
# Test PBC distance across boundary - CORRECTED TEST
print(f"Cell vectors:\n  v1 = {lat.cell_vectors[0]}\n  v2 = {lat.cell_vectors[1]}")
print(f"Cell area: {lat.get_cell_area():.2f}\n")

# Test 1: Two points that should wrap to the same location (distance = 0)
p1 = np.array([5.0, 3.0])
p2 = p1 + lat.cell_vectors[0] + lat.cell_vectors[1]  # one full cell away

dist = lat.minimum_image_distance(p1, p2)
print(f"Test 1 (same point via PBC):")
print(f"  Point 1: {p1}")
print(f"  Point 2: {p2}")
print(f"  PBC distance: {dist:.6f} (should be ~0.0)")

# Test 2: Points with known small separation
corner1 = np.array([0.1, 0.1])
# Point near corner1 but accessed via opposite side of cell
corner2 = corner1 - np.array([0.2, 0.2])  # 0.2 units back = wraps around from other side

direct_dist = np.linalg.norm(corner2 - corner1)
pbc_dist = lat.minimum_image_distance(corner1, corner2)

print(f"\nTest 2 (small separation via PBC):")
print(f"  Point 1: {corner1}")
print(f"  Point 2: {corner2}")  
print(f"  Direct distance: {direct_dist:.4f}")
print(f"  PBC distance:    {pbc_dist:.4f}")
print(f"  Expected:        {np.linalg.norm([0.2, 0.2]):.4f}")

# Test 3: Use a small offset from cell vector
offset = np.array([0.15, 0.15])
p3 = offset
p4 = lat.cell_vectors[0] + lat.cell_vectors[1] - offset  # near opposite corner

direct = np.linalg.norm(p4 - p3)
pbc = lat.minimum_image_distance(p3, p4)

print(f"\nTest 3 (corners of cell):")
print(f"  Expected separation: {np.linalg.norm(2*offset):.4f}")
print(f"  Direct distance:     {direct:.4f}")
print(f"  PBC distance:        {pbc:.4f}")

Cell vectors:
  v1 = [112.8454   0.    ]
  v2 = [56.4227    97.7269831]
Cell area: 11028.04

Test 1 (same point via PBC):
  Point 1: [5. 3.]
  Point 2: [174.2681    100.7269831]
  PBC distance: 0.000000 (should be ~0.0)

Test 2 (small separation via PBC):
  Point 1: [0.1 0.1]
  Point 2: [-0.1 -0.1]
  Direct distance: 0.2828
  PBC distance:    0.2828
  Expected:        0.2828

Test 3 (corners of cell):
  Expected separation: 0.4243
  Direct distance:     195.0442
  PBC distance:        0.4243


In [ ]:
# Test minimum image distance
# Take two sites and calculate distance directly vs with PBC
site1_coord = lat.coordinates[0]
site2_coord = lat.coordinates[1]

# Direct distance (may be wrong if crosses boundary)
direct_dist = np.linalg.norm(site2_coord - site1_coord)

# Minimum image distance (correct with PBC)
pbc_dist = lat.minimum_image_distance(site1_coord, site2_coord)

print(f"Site 0 coord: {site1_coord}")
print(f"Site 1 coord: {site2_coord}")
print(f"Direct distance:  {direct_dist:.4f}")
print(f"PBC distance:     {pbc_dist:.4f}")
print(f"Should be ~1nn:   {lat.get_nn_distance(1):.4f}")

In [ ]:
# Test PBC wrapping - take a coordinate outside the box
test_coord = lat.coordinates[0] + lat.cell_vectors[0] * 1.5 + lat.cell_vectors[1] * 0.7
print(f"Original coordinate: {test_coord}")

wrapped = lat.apply_pbc(test_coord)
print(f"Wrapped coordinate:  {wrapped}")
print(f"Should be inside box: {np.all(wrapped >= 0) and np.all(wrapped < np.max(lat.cell_vectors))}")

In [ ]:
# Test nearest neighbor distances
print("Nearest neighbor distances:")
for order in range(1, 6):
    nn_dist = lat.get_nn_distance(order)
    print(f"  {order}nn: {nn_dist:.4f}")

In [ ]:
# Test lattice representation and cell area
print(f"Lattice: {repr(lat)}")
print(f"\nCell vectors:\n{lat.cell_vectors}")
print(f"Cell area: {lat.get_cell_area():.4f}")

## Test lattice PBC methods